In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
import re

In [9]:
driver = webdriver.Chrome()

In [10]:
#主キー：日本語名
#入れ子の辞書：
    #JpName
    #H
    #A
    #B
    #C
    #D
    #S
    #imageLink
    #type
    #rememberedMoves
    #characteristic
pokemon = {}

## GameWithからのスクレイピング

In [11]:

url = "https://gamewith.jp/pokemon-sv/article/show/375426"

driver.get(url)
html = driver.page_source

In [12]:
main_soup = BeautifulSoup(html)

In [13]:
import time

#名前を取得
def get_name(soup):
    title = soup.find("h3").text
    try:
        pattern = '(.*)の色違いと基本情報'
        name = re.match(pattern, title).group(1)
    except:
        pattern = '(.*)の基本情報'
        name = re.match(pattern, title).group(1)
    
    return name

#種族値を取得    
def get_base(soup):
    table = soup.find("div", class_="pokesv_bar_table")
    bars = table.find_all("span", class_="bar1")

    return (bars[0].text, bars[1].text, bars[2].text, bars[3].text, bars[4].text, bars[5].text)

#画像リンクを取得
def get_image_link(soup):
    image_div = soup.find("div", class_="js-pokemon-deff-color-switch")
    image_url = image_div.find("img").get("src")

    return image_url

#タイプを取得
def get_types(soup):
    type_span = soup.find("span", class_ = "inline")
    imgs = type_span.find_all("img")
    types = set()
    for img in imgs:
        types.add(img.get("alt"))
    return list(types)

#覚える技を取得
def get_moves(soup):
    moves_list = []
    skill_list = soup.find("ol", id="SkillList")
    moves = skill_list.find_all("li", class_="w-pksv-skill-list-element")
    for move in moves:
        move_name = move.find("div", class_="_name").find("a").text
        moves_list.append(move_name)

    return moves_list

#特性を取得
def get_charac(soup): 
    charac_list = []

    toku_list = soup.find("ol", class_="pksv_toku_list").find_all("li")

    for toku in toku_list:
        charac_name = toku.find("div", class_="_name").text
        if charac_name != "夢特性なし":
            charac_list.append(charac_name)
    
    return charac_list


In [14]:
pokemon_list = main_soup.find_all("li", class_="w-pksv-pokemon-list-element")

for element in pokemon_list:
    print("==============")
    
    #各ページにアクセス
    individual_url = element.find("a", class_="_name").get("href")
    
    driver.set_page_load_timeout(10)

    while (True):
        try:
            driver.get(individual_url)
            print("got")
            break
        except:
            print("time out")
            continue

    #画像取得できるまで繰り返し
    while(True):
        print("try load html")

        html = driver.page_source

        soup = BeautifulSoup(html)

        image_link = get_image_link(soup)
        if image_link != "https://img.gamewith.jp/assets/images/common/transparent1px.png":
            break

        driver.execute_script("window.scrollBy(0, 200);")

    print("html loaded")

    #日本語名
    name = get_name(soup)

    pokemon[name] = {}
    pokemon[name]["JpName"] = name

    #種族値
    (pokemon[name]["H"], pokemon[name]["A"], pokemon[name]["B"], pokemon[name]["C"], pokemon[name]["D"], pokemon[name]["S"]) = get_base(soup)

    #画像リンク
    pokemon[name]["imageLink"] = image_link

    #タイプ
    pokemon[name]["types"] = get_types(soup)

    #覚える技
    pokemon[name]["rememberedMoves"] = get_moves(soup)

    #特性
    pokemon[name]["characteristics"] = get_charac(soup)

    print(name)


got
try load html
html loaded
ヒトカゲ
got
try load html
html loaded
リザード
got
try load html
html loaded
リザードン
got
try load html
html loaded
ガラルニャース
got
try load html
html loaded
ウパー
got
try load html
html loaded
ヌオー
got
try load html
try load html
html loaded
ニャイキング
got
try load html
html loaded
ピカチュウ
got
try load html
html loaded
ライチュウ
got
try load html
try load html
try load html
html loaded
プリン
got
try load html
html loaded
プクリン
got
try load html
html loaded
コンパン
got
try load html
html loaded
モルフォン
got
try load html
html loaded
ディグダ
got
try load html
html loaded
ダグトリオ
got
try load html
html loaded
ニャース
got
try load html
html loaded
ペルシアン
got
try load html
html loaded
コダック
got
try load html
html loaded
ゴルダック
got
try load html
html loaded
マンキー
got
try load html
html loaded
オコリザル
got
try load html
html loaded
ガーディ
got
try load html
html loaded
ウインディ
got
try load html
html loaded
ヤドン
got
try load html
html loaded
ヤドラン
got
try load html
html loaded
コイル
got
try load html
html loaded
レアコイル
got

##　保存

In [15]:
import json

tf = open("data/pokemon.json", "w")
json.dump(pokemon, tf, ensure_ascii=False)
tf.close()